# Collect Dynamic Features

In [1]:
import preprocessingNA as pp
import data2D3D
import os
import glob
import numpy as np
import logging
import nibabel as nib
import gc
from skimage import transform
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
import tensorflow as tf
from torch.utils.data import SubsetRandomSampler
import torchvision
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import random
import pickle
import sys
%matplotlib inline
get_ipython().run_line_magic('matplotlib', 'inline')
import matplotlib.pyplot as plt
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from torch.utils import data as D
from torch import nn
from torch.nn import functional as F
from torchvision import models
import torch.optim as optim
from time import time
from data2D3D import data2D3D
from D_UNet import U_Net
from tqdm import tqdm
import pandas as pd
os.environ['KMP_DUPLICATE_LIB_OK']='True'



## Preprocessing

In [3]:
def normalise_image(image):
    '''
    make image zero mean and unit standard deviation
    '''

    img_o = np.float32(image.copy())
    m = np.mean(img_o)
    s = np.std(img_o)
    return np.divide((img_o - m), s)



def crop_or_pad_slice_to_size(slice, nx=224, ny=224):

    x, y = slice.shape

    x_s = (x - nx) // 2
    y_s = (y - ny) // 2
    x_c = (nx - x) // 2
    y_c = (ny - y) // 2

    if x > nx and y > ny:
        slice_cropped = slice[x_s:x_s + nx, y_s:y_s + ny]
    else:
        slice_cropped = np.zeros((nx, ny))
        if x <= nx and y > ny:
            slice_cropped[x_c:x_c + x, :] = slice[:, y_s:y_s + ny]
        elif x > nx and y <= ny:
            slice_cropped[:, y_c:y_c + y] = slice[x_s:x_s + nx, :]
        else:
            slice_cropped[x_c:x_c + x, y_c:y_c + y] = slice[:, :]

    return slice_cropped



##### 4D_image folders include all subjects' 4D images
##### Each subject created their own folders saved into 4D_image folder
##### In each subjects' folder, 3D images were created for each time steps


In [4]:
import fnmatch
def data_prepare4D (nx=224, norm=True, test=False, numS=1):
    num=0
    imageS=[]
    img_T=[]
    imageSS=[]
    filename3D=[]
    imgC=glob.glob(os.path.join('datacollect', 'patient???_frame??.nii.gz'))
    path='4D_image/patient'+'{:03d}'.format(numS)
    imgC=glob.glob(os.path.join(path, 'slice??.nii')) 
    imgC.sort()
    
    file_count = len(fnmatch.filter(os.listdir('4D_image/patient'+'{:03d}'.format(numS)), '*.nii'))
    stack=0

    for i in range(file_count):
        
        image_name=imgC[i]
        imageTemperate=nib.load(image_name).get_fdata()
        image=np.zeros((imageTemperate.shape[0],imageTemperate.shape[1],imageTemperate.shape[2]+2))
        image[:,:,1:-1]=imageTemperate
        image[:,:,0]=imageTemperate[:,:,0]
        image[:,:,-1]=imageTemperate[:,:,-1]
        
        imageS.append(image.shape[-1])
        #num=num+imageS[i]    
    
        target_resolution = (1.37, 1.37)      
        img = image
        img = normalise_image(img)
    
        n1_header = nib.load(image_name).header
        pixel_size=n1_header['pixdim'][1:3] 
        scale_vector = [pixel_size[0] / target_resolution[0], pixel_size[1] / target_resolution[1]]

        
        for zz in range(img.shape[2]):

            slice_img = np.squeeze(img[:, :, zz])
            slice_rescaled = transform.rescale(slice_img,scale_vector, order=1, 
                                               preserve_range=True,multichannel=False,mode = 'constant')

            slice_cropped = crop_or_pad_slice_to_size(slice_rescaled, nx, nx)
            img_T.append(slice_cropped)
  
            if zz!=0 and zz!=img.shape[2]-1:
                imageSS.append(stack)
                filename3D.append(image_name)
       
            stack=stack+1
    
    img_T=np.float32(np.array(img_T));    


    num=img_T.shape[0]    
    return img_T, imageSS, num

In [44]:
def collect4DMask(testloader, test_indices, test_tumor_dataset, numS, file_count):
    # load model
    unet_model = U_Net().to(device)
    checkpoint = torch.load('./saved_models/WCE/Basic_Unet_epoch_066.pth',map_location=torch.device('cpu'))
    unet_model.load_state_dict(checkpoint)
    # Testing process on test data.
    unet_model.eval()
    
    
    Maskvolume = np.zeros((nx,nx,file_count))
    fileIndex=1
    directory='4D_mask/patient'+'{:03d}'.format(numS)
    os.mkdir(directory)
    
    for example_index, data in enumerate(testloader):
        #print(example_index)
        image2D = data['image2D'].numpy()#.to(device)
        image3D = data['image3D'].numpy()#.to(device)        
        unet_model.eval()  
        image_tensor2D = torch.Tensor(image2D)
        image_tensor2D = image_tensor2D.view((1, 3, nx, nx)).to(device)
        image_tensor3D = torch.Tensor(image3D)
        image_tensor3D = image_tensor3D.view((1, 1, 3, nx, nx)).to(device)  
        output = unet_model(image_tensor2D, image_tensor3D).detach().cpu()
        output=F.softmax(output, dim=1)
        output = torch.max(output, 1)[1]
        output = output.numpy()
        output = np.resize(output, (nx, nx))
        Maskvolume[:,:,example_index%file_count]=output
    
        if (example_index+1)%file_count==0:
            img = nib.nifti1.Nifti1Image(Maskvolume, np.eye(4))
            img.get_data_dtype() == np.dtype(np.int16)
            file_name='slice'+'{:02d}'.format(fileIndex)
            img.to_filename(os.path.join(directory,file_name))
            fileIndex=fileIndex+1
            Maskvolume = np.zeros((nx,nx,file_count))

In [54]:
for i in tqdm(range(1,101)):
    path = '4D_image/patient' +'{:03d}'.format(i)
    nx=224;
    Test_img_T, Test_imageSS, Test_num=data_prepare4D(nx=224, norm=False, test=True, numS=i)
    test_tumor_dataset = data2D3D(path, Test_imageSS, Test_img_T, Test_img_T, Test_num, nx, aug=True)
    #test_indices=list(range(0,Test_num-2*file_count))

    #path, dirs, files = next(os.walk(path))
    file_count = len(glob.glob(os.path.join(path, 'slice??.nii'))) #len(files)
    number=np.int((Test_num-2*file_count))
    test_indices=list(range(0,number))

    test_sampler=torch.utils.data.SequentialSampler(test_indices)
    testloader = torch.utils.data.DataLoader(test_tumor_dataset, 1,num_workers=0, sampler=test_sampler, shuffle=False )

    image_name=glob.glob(os.path.join(path, 'slice01.nii')) 
    thickness=nib.load(image_name[0]).get_fdata().shape[-1]
    collect4DMask(testloader=testloader, test_indices=test_indices, test_tumor_dataset=test_tumor_dataset, numS=i, file_count=thickness)

100%|██████████| 3/3 [18:48<00:00, 376.14s/it]


<br>
<font size =5> Collect Dynamic Features

In [57]:
from scipy.stats import kurtosis
from scipy.stats import skew 

def dynamicFeature(file_count,numS,xResolution,yResolution,zResolution):
    TResolution=xResolution*yResolution*zResolution
    LVCvolume = []
    RVCvolume = []
    Myovolume = []
    for fileIndex in range(file_count):
        directory='4D_mask/patient'+'{:03d}'.format(numS)
        file_name=directory+'/slice'+'{:02d}'.format(fileIndex+1) +'.nii'
        image=nib.load(file_name).get_fdata()
        image1=image.copy()
        image1[(image1!=1)]=0
        image1[(image1==1)]=1
        #image1Agg=image1.sum(axis=2)*zResolution
        RVCvolume.append(TResolution*image1.sum(axis=0).sum(axis=0).sum(axis=0))

        image2=image.copy()
        image2[(image2!=2)]=0
        image2[(image2==2)]=1
        Myovolume.append(TResolution*image2.sum(axis=0).sum(axis=0).sum(axis=0))
        #image2Agg=image2.sum(axis=2)
        #IMG2AGG=(mask*image2).sum(axis=2)*zResolution

        image3=image.copy()
        image3[(image!=3)]=0
        image3[(image3==3)]=1
        LVCvolume.append(TResolution*image3.sum(axis=0).sum(axis=0).sum(axis=0))
        #image3Agg=image3.sum(axis=2)*zResolution
    RVCvolume=np.asarray((RVCvolume))
    Myovolume=np.asarray((Myovolume))
    LVCvolume=np.asarray((LVCvolume))
    vMaxRVC=np.max(RVCvolume)
    vMaxMyo=np.max(Myovolume)
    vMaxLVC=np.max(LVCvolume)
    vMinRVC=np.min(RVCvolume)
    vMinMyo=np.min(Myovolume)
    vMinLVC=np.min(LVCvolume)
    vMedRVC=np.median(RVCvolume)
    vMedMyo=np.median(Myovolume)
    vMedLVC=np.median(LVCvolume)
    vkurtosisRVC=kurtosis(RVCvolume)
    vkurtosisMyo=kurtosis(Myovolume)
    vkurtosisLVC=kurtosis(LVCvolume)
    vskewRVC=skew(RVCvolume)
    vskewMyo=skew(Myovolume)
    vskewLVC=skew(LVCvolume)
    vstdRVC=np.std(RVCvolume)
    vstdMyo=np.std(Myovolume)
    vstdLVC=np.std(LVCvolume)
    vminLVC_vminRVC=vMinLVC/vMinRVC
    vminLVM_vminLVC=vMinMyo/vMinLVC
    vminRVC_vminLVM=vMinRVC/vMinMyo

#time step difference t(vmin,LVC)-t(vmin,RVC) time step difference t(vmax,LVC)-t(vmax,RVC)


    return  vMaxRVC,vMaxMyo,vMaxLVC,vMinRVC,vMinMyo,vMinLVC,vMedRVC,vMedMyo,vMedLVC,\
    vkurtosisRVC,vkurtosisMyo,vkurtosisLVC,vskewRVC,vskewMyo,vskewLVC,vstdRVC,vstdMyo,vstdLVC,\
    vminLVC_vminRVC,vminLVM_vminLVC,vminRVC_vminLVM

    

In [65]:
vMaxRVC,vMaxMyo,vMaxLVC,vMinRVC,vMinMyo,vMinLVC,vMedRVC,vMedMyo,vMedLVC,\
vkurtosisRVC,vkurtosisMyo,vkurtosisLVC,vskewRVC,vskewMyo,vskewLVC,vstdRVC,vstdMyo,vstdLVC,\
vminLVC_vminRVC,vminLVM_vminLVC,vminRVC_vminLVM = [[] for i in range(21)]
for i in  tqdm(range(1,101)):
    image_name='4D_image/patient' +'{:03d}'.format(i)+'/slice01.nii'
    xResolution= 1.37 #nib.load(image_name).header['pixdim'][1]
    yResolution= 1.37 #nib.load(image_name).header['pixdim'][2]
    zResolution=nib.load(image_name).header['pixdim'][3]
    TResolution=xResolution*yResolution*zResolution
    path='4D_mask/patient' +'{:03d}'.format(i)
    file_count = len(glob.glob(os.path.join(path, 'slice??.nii')))
    VMaxRVC,VMaxMyo,VMaxLVC,VMinRVC,VMinMyo,VMinLVC,VMedRVC,VMedMyo,VMedLVC,\
    VkurtosisRVC,VkurtosisMyo,VkurtosisLVC,VskewRVC,VskewMyo,VskewLVC,VstdRVC,VstdMyo,VstdLVC,\
    VminLVC_vminRVC,VminLVM_vminLVC,VminRVC_vminLVM=dynamicFeature(file_count,i,xResolution,yResolution,zResolution)
    
    vMaxRVC.append(VMaxRVC)
    vMaxMyo.append(VMaxMyo)
    vMaxLVC.append(VMaxLVC)
    vMinRVC.append(VMinRVC)
    vMinMyo.append(VMinMyo)
    vMinLVC.append(VMinLVC)
    vMedRVC.append(VMedRVC)
    vMedMyo.append(VMedMyo)
    vMedLVC.append(VMedLVC)
    vkurtosisRVC.append(VkurtosisRVC)
    vkurtosisMyo.append(VkurtosisMyo)
    vkurtosisLVC.append(VkurtosisLVC)
    vskewRVC.append(VskewRVC)
    vskewMyo.append(VskewMyo)
    vskewLVC.append(VskewLVC)
    vstdRVC.append(VstdRVC)
    vstdMyo.append(VstdMyo)
    vstdLVC.append(VstdLVC)
    vminLVC_vminRVC.append(VminLVC_vminRVC)
    vminLVM_vminLVC.append(VminLVM_vminLVC)
    vminRVC_vminLVM.append(VminRVC_vminLVM)
    

100%|██████████| 100/100 [03:35<00:00,  2.16s/it]


<br>
<font size =5> Merge the dynamic features with the instant features

In [74]:
dataset = pd.read_csv('featureRecords.csv')
y["Group"]=dataset["Group"]
dataset.drop(["Group"],axis=1, inplace=True)
dataset["vMaxRVC"]=vMaxRVC
dataset["vMaxMyo"]=vMaxMyo
dataset["vMaxLVC"]=vMaxLVC
dataset["vMinRVC"]=vMinRVC
dataset["vMinMyo"]=vMinMyo
dataset["vMinLVC"]=vMinLVC
dataset["vMedRVC"]=vMedRVC
dataset["vMedMyo"]=vMedMyo
dataset["vMedLVC"]=vMedLVC
dataset["vkurtosisRVC"]=vkurtosisRVC
dataset["vkurtosisMyo"]=vkurtosisMyo
dataset["vkurtosisLVC"]=vkurtosisLVC
dataset["vskewRVC"]=vskewRVC
dataset["vskewMyo"]=vskewMyo
dataset["vskewLVC"]=vskewLVC
dataset["vstdRVC"]=vstdRVC
dataset["vstdMyo"]=vstdMyo
dataset["vstdLVC"]=vstdLVC
dataset["vminLVC_vminRVC"]=vminLVC_vminRVC
dataset["vminLVM_vminLVC"]=vminLVM_vminLVC
dataset["vminRVC_vminLVM"]=vminRVC_vminLVM
dataset["Group"]=y["Group"]

In [84]:
dataset.to_csv("allFeatures.csv")